In [2]:
import os
import librosa
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity


def extract_mfcc(file_path, n_mfcc=13):
    """Extract MFCCs from an audio file."""
    y, sr = librosa.load(file_path, sr=None)
    mfcc = librosa.feature.mfcc(y=y, sr=sr, n_mfcc=n_mfcc)
    mfcc_mean = np.mean(mfcc, axis=1)  # Use mean vector for comparison
    return mfcc_mean


def enroll_speakers(enroll_dir):
    """Enroll each speaker from their audio files."""
    speaker_profiles = {}
    for speaker_name in os.listdir(enroll_dir):
        speaker_path = os.path.join(enroll_dir, speaker_name)
        if os.path.isdir(speaker_path):
            mfccs = []
            for file in os.listdir(speaker_path):
                if file.endswith(".wav"):
                    path = os.path.join(speaker_path, file)
                    mfcc = extract_mfcc(path)
                    mfccs.append(mfcc)
            speaker_profiles[speaker_name] = np.mean(mfccs, axis=0)
    return speaker_profiles


def identify_speaker(test_file, speaker_profiles):
    """Identify speaker by comparing test audio with enrolled profiles."""
    test_mfcc = extract_mfcc(test_file)
    similarities = {}
    for speaker, profile_mfcc in speaker_profiles.items():
        similarity = cosine_similarity([test_mfcc], [profile_mfcc])[0][0]
        similarities[speaker] = similarity
    best_match = max(similarities, key=similarities.get)
    return best_match, similarities[best_match]


# Example Usage

# 1. Folder structure for enrollment:
# enroll/
# ├── alice/
# │   ├── 1.wav
# │   └── 2.wav
# ├── bob/
# │   ├── 1.wav
# │   └── 2.wav

# 2. Test file: test.wav

enroll_dir = "enroll"
test_file = "test.wav"

speaker_profiles = enroll_speakers(enroll_dir)
predicted_speaker, score = identify_speaker(test_file, speaker_profiles)

print(f"Identified speaker: {predicted_speaker} (cosine similarity: {score:.4f})")

FileNotFoundError: [Errno 2] No such file or directory: 'enroll'